In [ ]:
from keras.applications import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
train_path = 'dataset/train'
test_path = 'dataset/test'
val_path = 'dataset/val'

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1/255)
test_data_gen = ImageDataGenerator(rescale=1/255)
val_data_gen = ImageDataGenerator(rescale=1/255)


In [ ]:
train_generator = train_data_gen.flow_from_directory(
        train_path,
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical')

In [ ]:
test_generator = test_data_gen.flow_from_directory(
    test_path,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
val_generator = val_data_gen.flow_from_directory(
    val_path,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'

)

In [ ]:
checkpoint = ModelCheckpoint(
    f'inceptionv3.h5',
     monitor='val_accuracy',
     verbose=1,
     save_best_only=True,
     mode = 'max'
)

In [ ]:
earlystop = EarlyStopping(monitor='val_accuracy',
                          patience=5,
                          verbose=1,
                          mode = 'max')

In [ ]:
inceptionv3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

model = Sequential()
model.add(inceptionv3)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Bazı katmanların eğitilmesini dondurmak (transfer öğrenme)
for layer in inceptionv3.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
result = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=15
)


In [ ]:
model.save("inceptionv3_model.keras")

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")



12/12 [==============================] - 163s 14s/step - loss: 0.1177 - accuracy: 0.9587
Test Loss: 0.11765893548727036
Test Accuracy: 0.9586777091026306
